In [1]:
from google.colab import files

files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"aniruddhpremdas1709","key":"ac6951a797ef9196b5aa550f4ca750b1"}'}

In [2]:
! mkdir ~/.kaggle
!mv /content/kaggle.json ~/.kaggle/

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [3]:
! chmod 600 ~/.kaggle/kaggle.json

In [4]:
! kaggle datasets download -d cdart99/food20dataset

food20dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [5]:
!unzip "/content/food20dataset.zip" -d "/content/"

Archive:  /content/food20dataset.zip
replace /content/food20dataset/test_set/biriyani/biriyanitest (1).jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: /content/food20dataset/test_set/biriyani/biriyanitest (1).jpg  
  inflating: /content/food20dataset/test_set/biriyani/biriyanitest (10).jpg  
  inflating: /content/food20dataset/test_set/biriyani/biriyanitest (11).jpg  
  inflating: /content/food20dataset/test_set/biriyani/biriyanitest (12).jpg  
  inflating: /content/food20dataset/test_set/biriyani/biriyanitest (13).jpg  
  inflating: /content/food20dataset/test_set/biriyani/biriyanitest (14).jpg  
  inflating: /content/food20dataset/test_set/biriyani/biriyanitest (15).jpg  
  inflating: /content/food20dataset/test_set/biriyani/biriyanitest (16).jpg  
  inflating: /content/food20dataset/test_set/biriyani/biriyanitest (17).jpg  
  inflating: /content/food20dataset/test_set/biriyani/biriyanitest (18).jpg  
  inflating: /content/food20dataset/test_set/biriyani/biriyanitest (19).j

In [44]:

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torchvision.datasets import ImageFolder


In [190]:

data_dir='/content/food20dataset/'
transform=transforms.Compose([
                              transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
                              transforms.RandomRotation(degrees=15),
                              transforms.ColorJitter(),
                              transforms.RandomHorizontalFlip(),
                              transforms.CenterCrop(size=224),  # Image net standards
                              transforms.ToTensor(),
                              transforms.Normalize([0.485, 0.456, 0.406],
                                                   [0.229, 0.224, 0.225])
                              ])        

test_transform=transforms.Compose([
                              transforms.Resize(size=256),
                              transforms.CenterCrop(size=224),
                              transforms.ToTensor(),
                              transforms.Normalize([0.485, 0.456, 0.406],
                                                   [0.229, 0.224, 0.225])
                              ])        

             
train_set=ImageFolder(data_dir+'/train_set',transform=transform)
test_set=ImageFolder(data_dir+'/test_set',transform=test_transform)

In [46]:
print(len(train_set))
print(train_set.classes)
print(len(test_set.classes))

1400
['biriyani', 'bisibelebath', 'butternaan', 'chaat', 'chappati', 'dhokla', 'dosa', 'gulab jamun', 'halwa', 'idly', 'kathi roll', 'meduvadai', 'noodles', 'paniyaram', 'poori', 'samosa', 'tandoori chicken', 'upma', 'vada pav', 'ven pongal']
20


In [47]:
import os


from torch.utils.data import DataLoader

In [136]:
classes = sorted(os.listdir(data_dir + "/train_set"))
classes

['biriyani',
 'bisibelebath',
 'butternaan',
 'chaat',
 'chappati',
 'dhokla',
 'dosa',
 'gulab jamun',
 'halwa',
 'idly',
 'kathi roll',
 'meduvadai',
 'noodles',
 'paniyaram',
 'poori',
 'samosa',
 'tandoori chicken',
 'upma',
 'vada pav',
 'ven pongal']

In [49]:
print(type(train_set[200][0]))
img,label=train_set[2]
print(img.shape)
print(label)

<class 'torch.Tensor'>
torch.Size([3, 224, 224])
0


In [18]:
random_seed=20
torch.manual_seed(random_seed)

In [19]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [20]:
batch_size = 64

train_loader=DataLoader(
    train_set,
    batch_size=batch_size, shuffle=True
)
test_loader=DataLoader(
    test_set,
    batch_size=int(batch_size/2), shuffle=True
)
# for images , labels in enumerate(train_loader):
#   print(labels)

In [21]:
from torch.optim import Adam
from torch.autograd import Variable

In [22]:
model = models.vgg16_bn(pretrained=True)
# Freeze model weights
for param in model.parameters():
    param.requires_grad = False

In [23]:
model.classifier[6] = nn.Sequential(
                      nn.Linear(4096, 256), 
                      nn.ReLU(), 
                      nn.Dropout(0.5),
                      nn.Linear(256, 20),                   
                      nn.LogSoftmax(dim=1))
for param in model.classifier[6].parameters():
    param.requires_grad = True

In [24]:
model = model.to(device)

In [25]:
torch.cuda.is_available()

True

In [26]:

#Optmizer and loss function
optimizer=Adam(model.parameters(),lr=0.0005,weight_decay=0.00001)
loss_function=nn.NLLLoss()

In [27]:
train_count=len(train_set)
test_count=len(test_set)

In [28]:
print(train_count,test_count)

1400 600


In [29]:
#Model training and saving best model
num_epochs=30
best_accuracy=0.0

for epoch in range(num_epochs):
    
    #Evaluation and training on training dataset
    model.train()
    
    train_accuracy=0.0
    train_loss=0.0
    
    for i, (images,labels) in enumerate(train_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        optimizer.zero_grad()
        
        outputs=model(images)
        loss=loss_function(outputs,labels)
        loss.backward()
        optimizer.step()
        
        
        train_loss+= loss.cpu().data*images.size(0)
        _,prediction=torch.max(outputs.data,1)
        
        train_accuracy+=int(torch.sum(prediction==labels.data))
        
    train_accuracy=train_accuracy/train_count
    train_loss=train_loss/train_count
    
    
    # Evaluation on testing dataset
    model.eval()
    test_accuracy=0.0
    for i, (images,labels) in enumerate(test_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        outputs=model(images)
        _,prediction=torch.max(outputs.data,1)
        test_accuracy+=int(torch.sum(prediction==labels.data))
    
    test_accuracy=test_accuracy/test_count
    
    
    print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_accuracy)+' Test Accuracy: '+str(test_accuracy))
    
    #Save the best model
    if test_accuracy>best_accuracy:
        torch.save(model.state_dict(),'best_checkpoint.model')
        best_accuracy=test_accuracy

Epoch: 0 Train Loss: tensor(2.5782) Train Accuracy: 0.27785714285714286 Test Accuracy: 0.67
Epoch: 1 Train Loss: tensor(1.6699) Train Accuracy: 0.56 Test Accuracy: 0.7266666666666667
Epoch: 2 Train Loss: tensor(1.2910) Train Accuracy: 0.6378571428571429 Test Accuracy: 0.7366666666666667
Epoch: 3 Train Loss: tensor(1.0750) Train Accuracy: 0.7085714285714285 Test Accuracy: 0.7733333333333333
Epoch: 4 Train Loss: tensor(0.9600) Train Accuracy: 0.7278571428571429 Test Accuracy: 0.7733333333333333
Epoch: 5 Train Loss: tensor(0.9180) Train Accuracy: 0.7264285714285714 Test Accuracy: 0.7983333333333333
Epoch: 6 Train Loss: tensor(0.7887) Train Accuracy: 0.7721428571428571 Test Accuracy: 0.7883333333333333
Epoch: 7 Train Loss: tensor(0.7613) Train Accuracy: 0.7678571428571429 Test Accuracy: 0.8066666666666666
Epoch: 8 Train Loss: tensor(0.7401) Train Accuracy: 0.7792857142857142 Test Accuracy: 0.82
Epoch: 9 Train Loss: tensor(0.7096) Train Accuracy: 0.78 Test Accuracy: 0.8083333333333333
Epoch

In [212]:
def predict_image(image):
    image_tensor = test_transform(image).float()
    image_tensor = image_tensor.unsqueeze_(0)
    input = Variable(image_tensor)
    input = input.to(device)
    output = model(input)
    index = output.data.cpu().numpy().argmax()
    pred=classes[index]
    return pred,index

In [ ]:
print(predict_image(im))



In [ ]:
fig=plt.figure(figsize=(10,10))
plt.axis('off')
image1=test_set[300][0]

image=image1.squeeze().permute(1,2,0)
plt.imshow(image)


##Shifting to GPU/CPU

##Training the Model

##Testing the model

##Saving the model